In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import pandas

seed = 7
np.random.seed(seed)
def string2bits(s=''):
    return [bin(ord(x))[2:].zfill(8) for x in s]
def bits2string(b=None):
    return ''.join([chr(int(x, 2)) for x in b])
dataframe = pandas.read_csv("hash_md5_dataset_sample.csv", header=None)
dataset = dataframe.values
x = dataset[:, 0:2]
X = []
Y= []
x_test=[]
y_test=[]

for i in range(45000):
    X.append(((''.join(string2bits(x[i][0])))))

for i in range(45000):
    Y.append(((''.join(string2bits(x[i][1])))))

for i in range(45000,50000):
    x_test.append(((''.join(string2bits(x[i][0])))))

for i in range(45000,50000):
    y_test.append(((''.join(string2bits(x[i][1])))))


x_train = np.array([[int(x) for x in y] for y in X])
y_train = np.array([[int(x) for x in y] for y in Y])

x_test = np.array([[int(x) for x in y] for y in x_test])
y_test = np.array([[int(x) for x in y] for y in y_test])

model = Sequential()
model.add(Dense(256, input_dim=256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(40,activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()
model.fit(x_train, y_train,
          epochs=200,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=1024, verbose=1)
classes = model.predict(x_test, batch_size=128)
print(score)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_17 (Dense)             (None, 512)               131584    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 40)                20520     
Total params: 217,896
Trainable params: 217,896
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
45000/45000 [==============================] - 0s - loss: 0.5743 - acc: 0.6340     
Epoch 2/200
45000/45000 [==============================] - 0s - loss: 0.5704 - acc: 0.6380     
Epoch 3/200
45000/45000 [==============================] - 0s - loss: 0.5698

In [49]:
classes[classes > 0.5] = 1
classes[classes <= 0.5]=0
classes.shape


(1000, 40)

In [58]:
values=[]
temp=''
z=0
for i in range(1000):
    for k in range(5):
        for j in range(8):
            temp=temp+str(int(classes[i][z]))
            z=z+1
        values.append(temp)
        temp=''
    z=0
#print values
print bits2string(values)

JiQ1baSQrQNpHrkmrJ|HDpgpTK0lzIJJZUYgqP0TKfr~4VxA3SgixUfXqPEYRUvtZeTSfKCsCpmu1Q:n[@5RGptxs4MqBvD}SOOxjqKhwcwwkt3R0mfR2p@ElX3u^i\xyJ`nyvSkhxTJQsxqAIywx^<5FqFuy|VvrPRpCsUptzUldIFpA0vujj4zbJCrcq01wMccC3NFfGtJiaI2pwv0fyBhXw0pgP:vH4QRxGJTrRm1ts1mPTuyegbq{EqXfHIlNMZExgBvqHJhFFVNxRV7PgYVsmrvgqR1UEWqRTu[ueqinJxXeaDsHvqdwTkfReHqEy1vDGyPbSKVrxoATTvew`qHDjROSC^qSNjSlmH2N2DWBS1InPr8X5sJ}XhxG\u8KfjNsGBBegvKjXH7Iq9IH6~[ByF2kvpBsCChmusUjSrtgPJl2TjMtqcSGPqFt|FtBmvyww`vnFi3WuiDdrQpBRCupYqjw9AWXpqJeJ|iGthrUJr[vRXhF~sJ0KtvDUu2mecS9r4pOwsaERt4GuusygAHJ4Wpdpru2ns2t0v0NuqP1sO@ZLxRqd2LJvuVPTwRzqUqMrzmoYzq[wGqVZPRa5tgRxWSDCWIT0[pTSAQftc2GxFIqE{9cxVFpC`Rs1}FXy^tEOiXCcs6Lnqf>hEerguXrZ{qExqtYsxS5GcelbrPsvYxer7p9rgLxptExp2Y}r^qCVvu{qoaaHX}KpHTFKsFafmYZs4LRNGwAgrBEg@uQbJcfqYuQnXKcvjUaKRTtCDZLw0rRgFdrDHakpMrDxY3AdulKSCTb:)TzRwviJW{rtRIjYIxxj4bpj2tB:v;wbrSsALearJ1r7sIgJt71QRAAiS6v9vG4YhjQGj0BvpJpL`G3PSWylgr7xh6@FpjseGzrtiEFqz8riC26eRIoreQUVAtWpWUGhSie{mI:VCtXcdrIT8mRqly0E7RmIusD4Ort7vxMn1ygfI:DfGpqUdUaifk21ZppU6Z

array([ 0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.], dtype=float32)

emXrMDcjMjgxT0aoBZ4dkfBHhh9iKKghXJzj7yL89tpEKa1uCo1NevGpD61b5GX6Yu6XGejnjXXNWEIieSPOquyuDXPxpy53LJAmTYwEJaR9sx4b7wtu8f12RPudJhLzLQa3TdY6hxOk1ZpW46xP4yXuAcyJZTshJknHNJI8zpiZgAUAoaa7LNfGuVtaA6W4JmE3Bj0YmAUw7aEMTImKq9xVOP0R8CQI4TJDXTCklwPi8tVgzM6sNrCA4cDRFSThiUrbv9VKC4rQPVdCeCDfOTEmqXCRfdn8WyoH2TsfgLNyABsytKV02Cbb4HB7YmSeL8CcSdxoep84knNY6oEP0QjQfj9hzD7kd8I6jqrZKoHh0I7kq5KU0UfYHzTDS30nzg36clFsiyxvkRcUAbDnTzLqeObiTSfkjaxNMl28Dg5QdJvQtEdtLBYTyPhwtwprlgEOLBM5W0AwxSQEGPav2z8uMrzCnMgTQcGU2EcWPUDOe5eAAKs100rJLSfiF8IhkNwLLUrRGezvVV7XXMNtZwF02W53x0SFNUxQRI3giQFZoQNiSonBqHSNnXYfgl8zUqghIK5OhzpCzO5bl9olUCkbwXq0gPI2fU57RhficgowRkdoShNK0EFVzNwBhBXZ1oSJ8HJtcPUgSzThrDfbbzEK2ja5wMimv3xKkoSuy8JhxfanrYjGs4dCIy980sXenvx3sAoChDTWbyj0WdVXrCzFoBBuDQaPXd34b2TLK2ykpBscDFBz2BDRZkiDoNXQHPwXUQ4qyrMdBjq2yYnB9KfrVLL8eBk3vXgsGO1KjyZwIMDejMYqI8JJWRG9XKMkgW6rDmwKGi8GDwgq6rWlFpipaBPzJqt5KTluNOguQO1mbf7EKd8n3kYr8Y82EFYltFSfzLqPnPcJFDvUs8iKEUesk2M4tnrtdNJqBt0cgTKZvrbqNim68KGKJGRFJnfiajiej2yLSSMLPXoEHGrosknH4JeZkfXgsIOyfHyb